# LightGBM + Dask

<table>
    <tr>
        <td>
            <img src="./_img/lightgbm.svg" width="300">
        </td>
        <td>
            <img src="./_img/dask-horizontal.svg" width="300">
        </td>
        <td>
            <img src="./_img/aws.svg" width="150">
        </td>
    </tr>
</table>

This notebook shows how to use `lightgbm.dask` to train a LightGBM model on data stored as a [Dask Array](https://docs.dask.org/en/latest/array.html). It uses `FargateCluster` from [`dask-cloudprovider`](https://github.com/dask/dask-cloudprovider) to create a distributed cluster running on [AWS Fargate](https://aws.amazon.com/fargate/).

To explore other topics in greater depth, see the other notebooks.

<hr>

## Set up a Dask cluster on AWS Fargate

Before running any of the code in the notebook, follow the instructions in ["Test with a FargateCluster"](../README.md##test-with-a-fargatecluster).

In [1]:
import json

with open("../ecr-details.json", "r") as f:
    ecr_details = json.loads(f.read())

CONTAINER_IMAGE = ecr_details["repository"]["repositoryUri"] + ":1"
print(f"scheduler and worker image: {CONTAINER_IMAGE}")

scheduler and worker image: public.ecr.aws/w8s1c8b1/dask-lgb-test-4468592b-5204-4e3b-ad80-7c5ae698472a-cluster:1


Before proceeding, set up your AWS credentials. If you're unsure how to do this, see [the AWS docs](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html).

In [2]:
import os

os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

Create a cluster with 3 workers. See https://cloudprovider.dask.org/en/latest/aws.html#dask_cloudprovider.aws.FargateCluster for more options.

In [7]:
from dask.distributed import Client
from dask_cloudprovider.aws import FargateCluster

n_workers = 3
cluster = FargateCluster(
    image=CONTAINER_IMAGE,
    worker_cpu=512,
    worker_mem=4096,
    n_workers=n_workers,
    fargate_use_private_ip=False,
    scheduler_timeout="40 minutes",
    find_address_timeout=60 * 10,
    worker_extra_args=["--nprocs=2"]
)
client = Client(cluster)
client.wait_for_workers(n_workers)

Exception ignored in: <function Cluster.__del__ at 0x7f2546f11ee0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/distributed/deploy/cluster.py", line 109, in __del__
    if self.status != Status.closed:
AttributeError: 'FargateCluster' object has no attribute 'status'
Exception ignored in: <function Cluster.__del__ at 0x7f2546f11ee0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/distributed/deploy/cluster.py", line 109, in __del__
    if self.status != Status.closed:
AttributeError: 'FargateCluster' object has no attribute 'status'
/opt/conda/lib/python3.8/contextlib.py:120: UserWarning: Creating your cluster is taking a surprisingly long time. This is likely due to pending resources on AWS. Hang tight! 
  next(self.gen)
/opt/conda/lib/python3.8/site-packages/distributed/client.py:1185: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     |

In [8]:
print(f"View the dashboard: {cluster.dashboard_link}")

View the dashboard: http://44.242.164.141:8787/status


Click the link above to view a diagnostic dashboard while you run the training code below.

<hr>

## Get some training data

This example uses `sklearn.datasets.make_regression()` to generate a dataset in `numpy` format, then uses `dask.Array.from_array()` to turn that into a Dask Array.

That's just done for convenience. `lightgbm.dask` just expects that your data are Dask Arrays or Dask DataFrames.

In [9]:
import dask.array as da
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=10000, random_state=42)
dX = da.from_array(X, chunks=(1000, X.shape[1]))
dy = da.from_array(y, chunks=1000)

Right now, the Dask Arrays `data` and `labels` are lazy. Before training, you can force the cluster to compute them by running `.persist()` and then wait for that computation to finish by `wait()`-ing on them.

Doing this is optional, but it will make data loading a one-time cost so subsequent runs are fast.

In [11]:
from dask.distributed import wait

dX = dX.persist()
dy = dy.persist()
_ = wait([dX, dy])

<hr>

## Train a model

With the data set up on the workers, train a model. `lightgbm.dask.DaskLGBMRegressor` has an interface that tries to stay as close as possible to the non-Dask scikit-learn interface to LightGBM (`lightgbm.sklearn.LGBMRegressor`).

In [12]:
from lightgbm.dask import DaskLGBMRegressor

dask_reg = DaskLGBMRegressor(
    client=client,
    max_depth=5,
    objective="regression_l1",
    learning_rate=0.1,
    tree_learner="data",
    n_estimators=10,
    min_child_samples=1,
)

dask_reg.fit(
    X=dX,
    y=dy,
)

/opt/conda/lib/python3.8/site-packages/lightgbm/dask.py:525: UserWarning: Parameter n_jobs will be ignored.
  _log_warning(f"Parameter {param_alias} will be ignored.")


Finding random open ports for workers


DaskLGBMRegressor(client=<Client: 'tcp://172.31.62.91:8786' processes=6 threads=6, memory=22.35 GiB>,
                  max_depth=5, min_child_samples=1, n_estimators=10,
                  num_threads=1, objective='regression_l1', time_out=120,
                  tree_learner='data')

In [14]:
dask_reg.machines

'172.31.63.34:37953,172.31.63.34:49763,172.31.5.137:43329,172.31.5.137:39591,172.31.2.25:48257,172.31.2.25:37347'

<hr>

## Evaluate the model

The `.predict()` method takes in a Dask collection and returns a Dask Array.

In [ ]:
preds = dask_reg.predict(dX)
print(str(preds))

Before calculating the mean absolute error (MAE) of these predictions, compute some summary statistics on the target variable. This is necessary to understand what "good" values of MAE look like.

In [ ]:
p = [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]
dy_percentiles = da.percentile(dy, p).compute()

for i, percentile in enumerate(p):
    print(f"{percentile * 100}%: {round(dy_percentiles[i], 2)}")

The metrics functions from `dask-ml` match those from `scikit-learn`, but take in and return Dask collections. You can use these functions to perform model evaluation without the evaluation data or predictions needing to be pulled down to the machine running this notebook. Pretty cool, right?

In [ ]:
from dask_ml.metrics.regression import mean_absolute_error

mean_absolute_error(preds, dy)

## Next Steps

Learn more: https://lightgbm.readthedocs.io/en/latest/Python-API.html#dask-api.

Ask a question, report a bug, or submit a feature request: https://github.com/microsoft/LightGBM/issues.

Contribute: https://github.com/microsoft/LightGBM/issues?q=is%3Aissue+is%3Aopen+label%3Adask.